# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание проекта

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
___
**Цель исследования:** построить модель для задачи классификации, которая выберет подходящий тариф.
___
**Этапы исследования:**
- [Откроем и изучим файл](#section1)
- [Разобьем данные на выборки](#section11)
- [Исследуем модели](#section2)
- [Проверим модели на тестовой выборке](#section3)
- [Проверим модели на адекватность](#section4)
- [Вывод](#section5)
___
**Описание данных**
- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

<a name="section1"></a>
## Откройте и изучите файл

In [13]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [14]:
try:
    data = pd.read_csv('users_behavior.csv', sep=',')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv', sep=',')

In [15]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [17]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [18]:
data.duplicated().sum()

0

Так как по условию предобработка данных уже была проведена - пропуски и дубликаты отсутствуют.

<a name="section11"></a>
## Разбейте данные на выборки

Введем переменные с признаками и целевым признаком

In [19]:
features = data.drop(['is_ultra'], axis=1)
target = data['is_ultra']

Разделим набор данных на обучающую (`features_train`), валидационную (`features_valid`) и тестовую выборки (`features_test`)

In [20]:
features_train, features_test, target_train, target_test = train_test_split(
    features,
    target,
    test_size=0.2,
    random_state=123,
    stratify=target)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train,
    target_train,
    test_size=0.25,
    random_state=123,
    stratify=target_train)

print('Обучающая выборка:', features_train.shape[0])
print('Валидационная выборка:', features_valid.shape[0])
print('Тестовая выборка:', features_test.shape[0])

Обучающая выборка: 1928
Валидационная выборка: 643
Тестовая выборка: 643


Разделили получившиеся выборки по соотношению 3:1:1

<a name="section2"></a>
## Исследуйте модели

### Дерево решений

In [21]:
model_tree = None
best_result = 0
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=123, max_depth=depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        model_tree = model
        best_result = result
        best_depth = depth

print('Модель с максимально большим значением accuracy:', best_result, 'Глубина дерева:', best_depth)

Модель с максимально большим значением accuracy: 0.7853810264385692 Глубина дерева: 3


### Случайный лес

In [22]:
model_forest = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1, 51, 5):
    for depth in range (1, 11):
        model = RandomForestClassifier(random_state=123, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train) 
        predictions_valid = model.predict(features_valid) 
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            model_forest = model
            best_result = result
            best_est = est
            best_depth = depth

print('Модель с максимально большим значением accuracy:', best_result, 'Глубина дерева:', best_depth, 'Количество деревьев:', best_est)

Модель с максимально большим значением accuracy: 0.80248833592535 Глубина дерева: 10 Количество деревьев: 21


### Логистическая регрессия

In [23]:
model_logistic = LogisticRegression(random_state=123, solver='lbfgs')
model_logistic.fit(features_train, target_train)
result = model_logistic.score(features_valid, target_valid)
print('Модель с максимально большим значением accuracy:',result)

Модель с максимально большим значением accuracy: 0.7527216174183515


***Вывод:*** мы провели исследования с тремя алгоритмами классификации и лучший результат показала модель `Случайный лес`: точность ~ **0.802**, глубина "дерева" - 10, количество "деревьев" - 21.

<a name="section3"></a>
## Проверьте модель на тестовой выборке

Так как на валидации лучше всех показала себя модель "Случайный лес", то прогоним ее и на тестовой выборке.

In [24]:
accuracy = model_forest.score(features_test, target_test)
print(f'Точность Случайного леса на тестовой выборке: {accuracy}')

Точность Случайного леса на тестовой выборке: 0.7993779160186625


**Вывод:** Проверка модели "Случайный лес" на тестовой выборке показала, что ее точность практически не уступает точности проверки на валидационной выборке.

<a name="section4"></a>
## (бонус) Проверьте модели на адекватность

Импортируем функцию `DummyClassifier`, чтобы проверить нашу модель на адекватность, сравнив метрику `accuracy`

In [25]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent", random_state=123)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6936236391912908

<a name="section5"></a>
## Общий вывод

Провели исследование трех моделей классификации и получили следующие результаты:
- Точность модели `Дерево решений` ~ 0.785
- Точность модели `Случайный лес` ~ **0.802**
- Точность модели `Логистическая регрессия` ~ 0.753

Модель `RandomForestClassifier` имеет наилучшую долю правильных ответов.
___
Точность `Случайного леса` на тестовой выборке - ~ 0.799. Эта модель на новых данных отработала практически не хуже, чем на валидационной выборке.
___

Точности всех трех моделей, которые мы исследовали выше, больше, чем точность `DummyClassifier` (~ 0.7). Это говорит о том, что `Дерево решений`, `Случайный лес` и `Логистическая регрессия` прошли проверку на адекватность.